In [1]:
!pip install google-generativeai pypdf requests transformers torch accelerate
import google.generativeai as genai
from google.colab import userdata, files
from pypdf import PdfReader
import requests
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import zipfile
import os
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.

In [2]:
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("HF_API_KEY")
login(token=api_key)


# Model setup with reliable model that works on Kaggle/Colab
model_name = "Qwen/Qwen2.5-1.5B-Instruct"  # More reliable and faster

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set pad token to avoid issues
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True
)

Loading tokenizer...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Loading model...


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

2025-07-30 13:30:16.995069: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753882217.253274      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753882217.329556      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [3]:
"""
def unzip_files(dataset_folder):
    for filename in os.listdir(dataset_folder):
        if filename.endswith(".zip"):
            filepath = os.path.join(dataset_folder, filename)
            try:
                with zipfile.ZipFile(filepath, 'r') as zip_ref:
                    zip_ref.extractall(dataset_folder)
                print(f"Successfully unzipped {filename}")
            except zipfile.BadZipFile:
                print(f"Error: {filename} is not a valid zip file")
            except Exception as e:
                print(f"An error occurred while unzipping {filename}: {e}")

# Replace 'dataset' with the actual path to your dataset folder if it's different
dataset_folder_path = '/content/dataset'
unzip_files(dataset_folder_path)
"""

'\ndef unzip_files(dataset_folder):\n    for filename in os.listdir(dataset_folder):\n        if filename.endswith(".zip"):\n            filepath = os.path.join(dataset_folder, filename)\n            try:\n                with zipfile.ZipFile(filepath, \'r\') as zip_ref:\n                    zip_ref.extractall(dataset_folder)\n                print(f"Successfully unzipped {filename}")\n            except zipfile.BadZipFile:\n                print(f"Error: {filename} is not a valid zip file")\n            except Exception as e:\n                print(f"An error occurred while unzipping {filename}: {e}")\n\n# Replace \'dataset\' with the actual path to your dataset folder if it\'s different\ndataset_folder_path = \'/content/dataset\'\nunzip_files(dataset_folder_path)\n'

In [4]:
RFP_PDF_file = "/kaggle/input/rfp-response-eval/1_CCTP.pdf"
RFP_response_file = "/kaggle/input/rfp-response-eval/rfp_response_standalone.pdf"

In [5]:
RFP_text_file = "" 
RFP_response_text_file =  ""

In [6]:
def read_RFP_file(PDF_file):
  text = ""
  reader = PdfReader(PDF_file)
  for page in reader.pages:
     text+= page.extract_text()
  return text



In [7]:
def get_prompt_template(RFP_file_text, RFP_response_text):
    """Create the evaluation prompt"""
    return f"""# RFP Response Evaluation Prompt

You are an expert in evaluating RFP responses. Your role is to analyze the quality of an RFP response against the specified requirements.

## Documents to analyze:
- **RFP Document**: {RFP_file_text[:2000]}...  # Truncated for processing
- **Proposed Response**: {RFP_response_text[:2000]}...  # Truncated for processing

## Evaluation Criteria:

### 1. Alignment with Requirements (25%)
- Does the response directly address the specific demands of the RFP?
- Are all mandatory points covered?
- Are there any missing elements or off-topic content?

### 2. Clarity and Structure (25%)
- Is the response coherent and well-organized?
- Is the proposed plan logical and easy to follow?
- Is the presentation professional and clear?

### 3. Cost and Value Proposition (25%)
- Is the proposed budget realistic and justified?
- Does the proposal offer good value for money?
- Are costs detailed and transparent?

### 4. Compliance with RFP Instructions (25%)
- Is the requested format respected?
- Are deadlines and constraints taken into account?
- Are all required documents provided?

## Required Response Format:

### Detailed Analysis:
For each criterion, provide:
- Specific analysis with examples drawn from the documents
- Identified strengths
- Observed gaps or weaknesses

### Final Score: X/10

### Recommendation:
**If score is < 7:**
List at least 3 specific improvements necessary for the response to be acceptable, with concrete implementation suggestions.

**If score is ≥ 7:**
Explain why this response deserves this high score by detailing its strengths and added value.

### Final Decision:
Clearly indicate whether you recommend submitting this response as-is, with minor modifications, or with major revisions.

---

**Important**: Your evaluation must be objective, constructive, and based solely on the provided documents. Give your reasoning step by step for each criterion before concluding.

Please provide your evaluation now:"""

In [8]:
"""!zip -r /content/dataset.zip /content/dataset
files.download("/content/dataset.zip")"""


'!zip -r /content/dataset.zip /content/dataset\nfiles.download("/content/dataset.zip")'

In [9]:
def generate_response(prompt, max_length=1500):
    """Generate response using the model with proper error handling"""
    try:
        # Create a more compatible prompt format for Qwen
        formatted_prompt = f"<|im_start|>system\nYou are an expert RFP evaluator.<|im_end|>\n<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"
        
        # Tokenize input with proper settings
        inputs = tokenizer(
            formatted_prompt, 
            return_tensors="pt", 
            max_length=3072,  # Leave room for generation
            truncation=True,
            padding=False
        )
        
        # Move inputs to the same device as model
        input_ids = inputs.input_ids.to(model.device)
        attention_mask = inputs.attention_mask.to(model.device)
        
        # Generate response with simpler parameters
        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=max_length,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                repetition_penalty=1.1,
                top_p=0.9
            )
        
        # Decode response
        full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Better cleaning - remove everything before "### Detailed Analysis:"
        if "### Detailed Analysis:" in full_response:
            response = "### Detailed Analysis:" + full_response.split("### Detailed Analysis:")[-1]
        elif "Detailed Analysis:" in full_response:
            response = "Detailed Analysis:" + full_response.split("Detailed Analysis:")[-1]
        else:
            # Fallback cleaning - remove common chat artifacts
            response = full_response
            # Remove system/user/assistant markers
            response = response.replace("system\nYou are an expert RFP evaluator.\nuser", "")
            response = response.replace("assistant\n", "")
            response = response.replace("system", "")
            response = response.replace("user", "")
            response = response.replace("assistant", "")
            # Remove the formatted prompt completely
            response = response.replace(formatted_prompt, "")
            
        return response.strip()
        
    except Exception as e:
        # Fallback: try with minimal parameters
        try:
            inputs = tokenizer(prompt[:2000], return_tensors="pt", truncation=True)
            inputs = {k: v.to(model.device) for k, v in inputs.items()}
            
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=800,
                    do_sample=False,  # Use greedy decoding as fallback
                    pad_token_id=tokenizer.eos_token_id
                )
            
            response = tokenizer.decode(outputs[0], skip_special_tokens=True)
            return response.replace(prompt[:2000], "").strip()
            
        except Exception as e2:
            return f"Error occurred during generation: {e2}"


In [10]:
def main():
    """Main execution function"""
    # Read PDF files
    RFP_text_file = read_RFP_file(RFP_PDF_file)
    RFP_response_text_file = read_RFP_file(RFP_response_file)
    
    if not RFP_text_file or not RFP_response_text_file:
        return
    
    # Generate evaluation prompt
    prompt = get_prompt_template(RFP_text_file, RFP_response_text_file)
    
    # Generate the evaluation
    verdict = generate_response(prompt)
    
    # Print ONLY the LLM output
    print("Done with verdict")
    with open("LLM_verdict.txt", "w") as f:
      f.write(verdict)
    print("Saved in txt")

# Execute the main function
if __name__ == "__main__":
    main()

Done with verdict
Saved in txt
